In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import math
import sys
sys.path.append('..')
import pyzx as zx
from pyzx.linalg import Mat2

In [2]:
m1 = Mat2([[1,0],
           [1,1]])
m2 = Mat2([[1,1],
           [1,1]])
m3 = Mat2([[1,0,1,1,0],
           [1,1,1,0,0],
           [1,1,0,0,1],
           [0,1,0,1,0],
           [0,0,1,1,0]])
m4 = Mat2([[1,0,1,0,0],
           [0,1,1,0,0],
           [1,1,0,0,1],
           [0,1,0,1,0],
           [0,0,0,1,1]])

In [3]:
result = Mat2([[1,1],[0,0]])
m1 * m2 == result

True

In [12]:
m3c = m3.copy()
m3c.gauss(full_reduce=True)
m3c.data[2][0] = 1
m3c.data[4][4] = 1
m3c

[ 1  0  0  0  0 ]
[ 0  1  0  0  1 ]
[ 1  0  1  0  1 ]
[ 0  0  0  1  1 ]
[ 0  0  0  0  1 ]

In [13]:
for i in range(m3c.rows()):
    for j in range(min(m3c.cols(),i)):
        if m3c.data[i][j]: print("AAAHHH")

AAAHHH


In [5]:
print('rank', m3.rank(), '<', m3.rows(), 'rows\n')
print(m3.inverse(), '\n')

rank 4 < 5 rows

None 



In [6]:
print('rank', m4.rank(), '=', m4.rows(), 'rows\n')
print(m4.inverse(), '\n')
print(m4 * m4.inverse(), '\n')
print(m4.inverse() * m4, '\n')

rank 5 = 5 rows

[ 0  0  1  1  1 ]
[ 1  1  1  1  1 ]
[ 1  0  1  1  1 ]
[ 1  1  1  0  1 ]
[ 1  1  1  0  0 ] 

[ 1  0  0  0  0 ]
[ 0  1  0  0  0 ]
[ 0  0  1  0  0 ]
[ 0  0  0  1  0 ]
[ 0  0  0  0  1 ] 

[ 1  0  0  0  0 ]
[ 0  1  0  0  0 ]
[ 0  0  1  0  0 ]
[ 0  0  0  1  0 ]
[ 0  0  0  0  1 ] 



In [20]:
b = Mat2([[1],[1],[1],[1],[1]])
x = m3.solve(b)
print('trying:\n' + str(b), '\n')
print('solution:\n' + str(x), '\n')

b = Mat2([[1],[1],[0],[0],[0]])
x = m3.solve(b)
print('trying:\n' + str(b), '\n')
print('solution:\n' + str(x), '\n')
print('checking:\n' + str(m3 * x), '\n')

trying:
[ 1 ]
[ 1 ]
[ 1 ]
[ 1 ]
[ 1 ] 

solution:
None 

trying:
[ 1 ]
[ 1 ]
[ 0 ]
[ 0 ]
[ 0 ] 

solution:
[ 1 ]
[ 1 ]
[ 1 ]
[ 1 ]
[ 0 ] 

checking:
[ 1 ]
[ 1 ]
[ 0 ]
[ 0 ]
[ 0 ] 



In [8]:
b = Mat2([[1],[0],[1],[1],[0]])
x = m4.solve(b)
m4 * x

[ 1 ]
[ 0 ]
[ 1 ]
[ 1 ]
[ 0 ]

In [7]:
mf0,mf1 = m3.factor()
print(mf0)
print('\n*\n')
print(mf1)
print('\n=\n')
print(mf0 * mf1)
print('\n==\n')
print(m3)

[ 1  0  0  0 ]
[ 1  1  0  0 ]
[ 1  1  1  0 ]
[ 0  1  0  0 ]
[ 0  0  1  1 ]

*

[ 1  0  1  1  0 ]
[ 0  1  0  1  0 ]
[ 0  0  1  0  1 ]
[ 0  0  0  1  1 ]

=

[ 1  0  1  1  0 ]
[ 1  1  1  0  0 ]
[ 1  1  0  0  1 ]
[ 0  1  0  1  0 ]
[ 0  0  1  1  0 ]

==

[ 1  0  1  1  0 ]
[ 1  1  1  0  0 ]
[ 1  1  0  0  1 ]
[ 0  1  0  1  0 ]
[ 0  0  1  1  0 ]


In [46]:
m = Mat2([[1, 0, 0, 0, 0, 1, 1, 0, 0, 0],
 [1, 1, 1, 1, 0, 1, 1, 1, 1, 1],
 [0, 1, 0, 1, 0, 0, 0, 1, 1, 1],
 [1, 1, 1, 1, 0, 0, 0, 1, 1, 1],
 [1, 0, 0, 1, 0, 0, 1, 0, 0, 0],
 [1, 0, 1, 0, 1, 0, 0, 0, 0, 0]])
#m = Mat2([[1, 0, 1, 0, 0], [1, 1, 1, 1, 1], [1, 1, 0, 1, 1], [1, 1, 0, 0, 0]])

In [28]:
def xor_rows(l1, l2):
    return [0 if l1[i]==l2[i] else 1 for i in range(len(l1))]

In [8]:
r = m.rows()
c = m.cols()
combs = {}
for i in range(r):
    for j in range(i+1,r):
        combs[(i,j)] = xor_rows(m.data[i],m.data[j])

In [42]:
def find_minimal_sums(m):
    r = m.rows()
    d = m.data
    combs = {(i,):d[i] for i in range(r)}
    while True:
        combs2 = {}
        for index,l in combs.items():
            for k in range(max(index)+1,r):
                row = xor_rows(combs[index],d[k])
                if sum(row) == 1:
                    return (*index,k), row
                combs2[(*index,k)] = row
        combs = combs2

def greedy_reduction(m):
    indices, row = find_minimal_sums(m)
    indices = list(indices)
    rows = {i:m.data[i] for i in indices}
    weights = {i: sum(r) for i,r in rows.items()}
    result = []
    while len(indices)>1:
        best = (0,1)
        reduction = -1
        for i in indices:
            for j in indices:
                if j <= i: continue
                w = sum(xor_rows(rows[i],rows[j]))
                if weights[i] - w > reduction:
                    best = (j,i) # "Add row j to i"
                    reduction = weights[i] - w
                if weights[j] - w > reduction:
                    best = (i,j)
                    reduction = weights[j] - w
        result.append(best)
        control, target = best
        rows[target] = xor_rows(rows[control],rows[target])
        weights[target] = weights[target] - reduction
        indices.remove(control)
    return result

In [18]:
combs3 = {}
for index in combs:
    for k in range(r):
        if any(i>k for i in index): continue
        combs3[(index[0],index[1],k)] = xor_rows(combs[index],m.data[k])

In [19]:
for index, l in combs3.items():
    if sum(l) == 1:
        print(index, l)

(0, 1, 2) [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
(2, 3, 5) [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
(0, 1, 3) [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [39]:
combs3[(*[1,2],3)]

[0, 1, 0, 1, 0, 1, 1, 1, 1, 1]

In [36]:
indices = (0,1,2)
combs = {(i,):m.data[i] for i in indices}
weights = {(i,): sum(m.data[i]) for i in indices}
for i in indices:
    for j in indices[i+1:]:
        row = xor_rows(m.data[i],m.data[j])
        combs[(i,j)] = row
        weights[(i,j)] = sum(row)
weights

{(0,): 3, (0, 1): 6, (0, 2): 8, (1,): 9, (1, 2): 4, (2,): 5}

In [57]:
class GaussPrinter:
    def row_add(self,i,j):
        print("row_add",i,j)

g = GaussPrinter()
m.gauss(x=g)
m

row_add 2 3
row_add 2 4
row_add 2 5
row_add 2 4
row_add 2 5
row_add 2 3


[ 1  0  0  0  0  1  1  0  0  0 ]
[ 0  1  1  1  0  0  0  1  1  1 ]
[ 0  0  1  0  0  0  0  0  0  0 ]
[ 0  0  0  1  0  0  1  0  0  0 ]
[ 0  0  0  0  1  0  0  0  0  0 ]
[ 0  0  0  0  0  1  1  0  0  0 ]

In [60]:
class CNOTPrinter:
    def row_add(self, r1,r2):
        print("row_add", r1,r2)

cn = CNOTPrinter()
m = Mat2.id(8)
m.data[0][7] = 1
print(m)
gauss(m,x=cn,blocksize=5,full_reduce=True)

[ 1  0  0  0  0  0  0  1 ]
[ 0  1  0  0  0  0  0  0 ]
[ 0  0  1  0  0  0  0  0 ]
[ 0  0  0  1  0  0  0  0 ]
[ 0  0  0  0  1  0  0  0 ]
[ 0  0  0  0  0  1  0  0 ]
[ 0  0  0  0  0  0  1  0 ]
[ 0  0  0  0  0  0  0  1 ]
row_add 7 0


8

In [80]:
def reduce_bottom_rows(m, qubits):
    cols = m.cols()
    leading_one = {}
    adds = []
    for r in range(qubits):
        while True:
            i = next(i for i in range(cols) if m.data[r][i])
            if i in leading_one:
                m.row_add(leading_one[i],r)
                adds.append((leading_one[i],r))
            else:
                leading_one[i] = r
                break
    for r in range(qubits, m.rows()):
        while True:
            if not any(m.data[r]): 
                return r
            i = next(i for i in range(cols) if m.data[r][i])
            if i not in leading_one: break
            m.row_add(leading_one[i], r)
            adds.append((leading_one[i],r))
    raise Exception("Did not find any completely reducable row")
    #return adds
print(reduce_bottom_rows(m, 10))
m

10


[ 0  0  0  0  0  0  0  0  0  1  0  1  0  1  0  0  0  1  1  0  1  1  0  1  0  0  0  0  0  0  0  0  1  0 ]
[ 0  0  0  0  0  0  0  0  0  0  1  1  0  0  0  0  0  1  1  1  0  0  0  0  0  0  0  0  0  0  1  0  0  0 ]
[ 0  0  0  0  0  0  0  0  0  0  0  0  1  1  1  0  0  0  0  1  1  1  0  1  0  0  0  0  0  0  1  0  0  0 ]
[ 1  1  1  1  1  1  0  1  0  1  0  0  1  1  0  1  1  0  0  1  1  1  1  0  1  1  0  1  1  1  0  1  1  1 ]
[ 0  1  1  1  1  1  1  1  1  1  1  1  0  0  0  0  0  0  0  0  1  1  1  0  1  1  1  0  0  1  1  0  1  0 ]
[ 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  1  1  1  0  0  0  0  0  0  0  0  0  0  0  0  1  0 ]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  1  1  1  0  1  0  0  0  0  0  0  0  0  1  0 ]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  1  1  0  0  0  0  0  0  0  0  0  0  1  0  1  0 ]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  1  0  0  0  0  0  0  0  0  0  0  0  0  1  0 ]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 

In [79]:
from pyzx.linalg import column_optimal_swap

s = """
[ 0  0  0  0  0  0  0  0  0  1  0  1  0  1  0  0  0  1  1  0  1  1  0  1  0  0  0  0  0  0  0  0  1  0 ]
[ 0  0  0  0  0  0  0  0  0  0  1  1  0  0  0  0  0  1  1  1  0  0  0  0  0  0  0  0  0  0  1  0  0  0 ]
[ 0  0  0  0  0  0  0  0  0  0  1  1  1  1  1  0  0  1  1  0  1  1  0  1  0  0  0  0  0  0  0  0  0  0 ]
[ 1  1  1  1  1  1  0  1  0  1  0  0  1  1  0  1  1  0  0  1  1  1  1  0  1  1  0  1  1  1  0  1  1  1 ]
[ 1  0  0  0  0  0  1  0  1  0  1  1  1  1  0  1  1  0  0  1  0  0  0  0  0  0  1  1  1  0  1  1  0  1 ]
[ 0  0  0  0  0  0  0  0  0  1  0  0  0  1  0  0  0  0  0  1  1  1  0  1  0  0  0  0  0  0  0  0  0  0 ]
[ 0  0  0  0  0  0  0  0  0  0  1  0  1  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ]
[ 0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  1  0  1  1  0  1  0  0  0  0  0  0  0  0  0  0 ]
[ 0  0  0  0  0  0  0  0  0  0  1  1  1  1  1  0  0  1  1  1  1  1  0  1  0  0  0  0  0  0  1  0  0  0 ]
[ 0  0  0  0  0  0  0  0  0  0  1  0  1  1  1  0  0  0  1  0  1  1  0  1  0  0  0  0  0  0  0  0  0  0 ]
[ 0  0  0  0  0  0  0  0  0  0  0  0  1  1  1  0  0  0  1  0  1  0  0  1  0  0  0  0  0  0  0  0  0  0 ]
[ 0  0  0  0  0  0  0  0  0  1  0  0  0  1  0  0  0  0  0  1  1  1  0  1  0  0  0  0  0  0  0  0  0  0 ]
[ 0  0  0  0  0  0  0  0  0  0  0  0  1  1  1  0  0  0  0  1  1  1  0  1  0  0  0  0  0  0  0  0  0  0 ]"""
l = s.splitlines()[1:]
l = [[int(i) for i in j[1:-1].split(" ") if i] for j in l]
m = Mat2(l)
#m.gauss()
#b = Mat2([[0],[0],[0],[0],[1],[0]])
#m.solve(b)
#m.gauss(full_reduce=True)
#m*Mat2([[0],[0],[1],[0]])
#column_optimal_swap(m)
cnots = m.to_cnots()
print([(cnot.target, cnot.control) for cnot in cnots])
# for cnot in cnots[:8]:
#     m.row_add(cnot.target,cnot.control)
#m.gauss(full_reduce=False)
#m

[(3, 0), (0, 3), (0, 4), (4, 1), (1, 4), (3, 5), (3, 12), (3, 2), (2, 3), (3, 9), (5, 12), (11, 13), (3, 4), (3, 6), (3, 10), (5, 4), (4, 5), (4, 6), (4, 10), (5, 6), (5, 7), (5, 10), (5, 11), (6, 7), (6, 8), (8, 10), (8, 11), (8, 13), (11, 13), (11, 10), (10, 11), (13, 11), (11, 13), (11, 9), (10, 7), (8, 6), (11, 5), (10, 4), (8, 2), (11, 10), (10, 7), (6, 5), (6, 3), (1, 0), (10, 6), (8, 7), (8, 5), (8, 2), (9, 1), (9, 8), (8, 0), (8, 4), (8, 6), (7, 2), (7, 3), (7, 4), (7, 5), (6, 0), (6, 1), (6, 4), (5, 0), (5, 2), (5, 3), (5, 4), (4, 1), (4, 3), (3, 2)]


In [91]:
circ = zx.generate.cliffordT(5, 400, p_t=0.2)
g = circ.copy()
zx.simplify.full_reduce(g)
c = zx.extract.streaming_extract(g, quiet=False)
zx.compare_tensors(circ.to_tensor(), c.to_tensor())

Greedy reduction with 1 CNOTs
Greedy reduction with 2 CNOTs
Complicated phase gadget
Greedy reduction with 1 CNOTs
Greedy reduction with 1 CNOTs
Greedy reduction with 1 CNOTs
Greedy reduction with 1 CNOTs
11/40
Greedy reduction with 2 CNOTs
Simple phase gadget
Greedy reduction with 1 CNOTs
Complicated phase gadget
Greedy reduction with 2 CNOTs
Greedy reduction with 1 CNOTs
Greedy reduction with 2 CNOTs
21/40
Greedy reduction with 2 CNOTs
Greedy reduction with 1 CNOTs
Greedy reduction with 1 CNOTs
Greedy reduction with 1 CNOTs
Greedy reduction with 1 CNOTs
31/40
Greedy reduction with 1 CNOTs
Greedy reduction with 1 CNOTs


True